**Task 09: Data linking**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2021-2022/master/Assignment4/course_materials/"

In [2]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"rdf/data03.rdf", format="xml")
g2.parse(github_storage+"rdf/data04.rdf", format="xml")

<Graph identifier=Nf00620a794cc47f7a10a2e9e2a36e9c2 (<class 'rdflib.graph.Graph'>)>

Busca individuos en los dos grafos y enlázalos mediante la propiedad OWL:sameAs, inserta estas coincidencias en g3. Consideramos dos individuos iguales si tienen el mismo apodo y nombre de familia. Ten en cuenta que las URI no tienen por qué ser iguales para un mismo individuo en los dos grafos.

In [3]:
def SPARQLqueries():
    q1 = prepareQuery("""
            SELECT ?valueName ?valueFamily
            WHERE {
                ?individual vcard:Given ?valueName;
                 vcard:Family ?valueFamily.
            }
            """, initNs={"vcard":"http://www.w3.org/2001/vcard-rdf/3.0#"})
    q2 = prepareQuery("""
            SELECT ?individual 
            WHERE {
                ?individual vcard:Given ?valueName;
                 vcard:Family ?valueFamily.
            }
            """, initNs={"vcard":"http://www.w3.org/2001/vcard-rdf/3.0#"})

   
    return q1,q2

In [4]:
for s,p,o in g1:
    print(s, p, o)

http://data.three.org#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.three.org#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.three.org#Person
http://data.three.org#SaraJones http://www.w3.org/2001/vcard-rdf/3.0#FN Sara Jones
http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://data.three.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Family Smith
http://data.three.org#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0#Given John
http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http:

In [5]:

for s,p,o in g2:
    print(s, p, o)

http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#FN Tim Berners Lee
http://data.four.org#0003 http://www.w3.org/2001/vcard-rdf/3.0#Family Potter
http://data.four.org#0003 http://www.w3.org/2001/vcard-rdf/3.0#FN Harry Potter
http://data.four.org#0005 http://www.w3.org/2001/vcard-rdf/3.0#FN John Doe
http://data.four.org#0003 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.four.org#Person
http://data.four.org#0001 http://www.w3.org/2001/vcard-rdf/3.0#Family Jones
http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#Given Tim
http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#Family Berners Lee
http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://data.four.org#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/200

In [6]:
for s,p,o in g3:
    print(s, p, o)

In [7]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import OWL

vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
ns1 = Namespace("http://data.three.org#")
ns2 = Namespace("http://data.four.org#")


g1Individuals = []
g2Individuals = []

q = SPARQLqueries()
print("Values in graph1: ")
for r1 in g1.query(q[0]):
    print("\t\t- "+r1.valueName+ " "+ r1.valueFamily)
    g1Individuals.append([r1.valueName, r1.valueFamily])

print("Values in graph2: ")
for r2 in g2.query(q[0]):
    print("\t\t- "+r2.valueName+" " +r2.valueFamily)
    g2Individuals.append([r2.valueName, r2.valueFamily])


comun_element = [element for element in g1Individuals if element in g2Individuals]
for element in comun_element:
    print("\nSame in both graphs: ")
    print(element[0]+" "+ element[1])
    
    for r3 in g1.query(q[1], initBindings={ "valueName":element[0], "valueFamily": element[1]}):
        print("\tGraph1 URI:")
        print(r3.individual)
    for r4 in g2.query(q[1], initBindings={ "valueName":element[0], "valueFamily": element[1]}):
        print("\tGraph2 URI:")
        print(r4.individual)
        
    g3.add((r3.individual,OWL.sameAs,r4.individual))
    

print("\n\tThe new graph3")
for s,p,o in g3:
    print(s, p, o)

Values in graph1: 
		- Sara Jones
		- John Smith
		- John Doe
Values in graph2: 
		- Sara Jones
		- Jonathan Smith
		- Harry Potter
		- Tim Berners Lee
		- John Doe

Same in both graphs: 
Sara Jones
	Graph1 URI:
http://data.three.org#SaraJones
	Graph2 URI:
http://data.four.org#0001

Same in both graphs: 
John Doe
	Graph1 URI:
http://data.three.org#JohnDoe
	Graph2 URI:
http://data.four.org#0005

	The new graph3
http://data.three.org#SaraJones http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0001
http://data.three.org#JohnDoe http://www.w3.org/2002/07/owl#sameAs http://data.four.org#0005
